In [11]:
import pandas as pd
import numpy as np
from statistics import mode


In [87]:
class Node(object):
    def __init__(self,ids=None,children=[],entropy=0,depth=0):
        self.ids=ids
        self.entropy=entropy
        self.depth=depth
        self.split_attribute=None # chosen feature to divide into children
        self.children=children
        self.val_chosen=None      # val_chosen to save values for predictation
        self.label=None

    def set_val(self,split_attribute,val_chosen):
        self.split_attribute=split_attribute
        self.val_chosen=val_chosen    # function to save 2 val for predictation

    def set_label(self,label):
        self.label=label

class decision_tree_id3(object):

    def __init__(self,max_depth=10,min_sample_split=2,min_gain=1e-4):
        self.root=None
        self.max_depth=max_depth
        self.min_sample_split=min_sample_split
        self.Ntrain=0
        self.min_gain=min_gain

    def optimizer_step(self,data,target):
        self.attributes = list(data)    # name features of data
        self.Ntrain=data.count().iloc[0]   # number of data
        self.data=data
        self.target=target
        self.labels=target.unique()     
        ids=range(self.Ntrain)
        self.root=Node(ids=ids,entropy=self.entropy(ids),depth=0)
        queue=[self.root]
        while queue:
            node=queue.pop()
            if node.depth<self.max_depth or node.entropy<self.min_gain:
                node.children=self._split(node)
                if not node.children:
                    self._set_label(node)
                queue+=node.children
            else:
                self._set_label(node) 

    def _split(self,node):
        ids=node.ids
        best_gain=0
        best_splits=[]
        best_attributes=None
        val_chosen=None
        
        sub_data=self.data.iloc[ids,:] # sub data of this node
        for i, att in enumerate(self.attributes):   
            values=sub_data.iloc[:,i].unique().tolist()
            if len(values)==1: continue
            splits=[]
            for value in values:
                sub_ids=sub_data.index[sub_data[att]==value].tolist()
                splits.append(sub_ids)
            if min(map(len,splits))<self.min_sample_split : continue    # map function apply len function for each element of splits
            entropy_=0
            for split in splits:
                entropy_+=len(split)*self.entropy(split)/len(ids)
            gain=node.entropy-entropy_
            if gain<self.min_gain: continue
            if gain>best_gain:
                best_gain=gain
                best_attributes=att
                best_splits=splits
                val_chosen=values
        node.set_val(best_attributes,val_chosen) 
        child_nodes=[Node(ids=split,entropy=self.entropy(split),depth=node.depth+1)for split in best_splits]
        return child_nodes

    def _set_label(self,node): 
        target_ids=node.ids
        node.set_label(self.target.iloc[target_ids].mode().iloc[0])

    def entropy(self,ids):
        if len(ids) == 0: return 0
        freq = np.array(self.target.iloc[ids].value_counts()) # the array with the number of the occurrences of each value                  
        freq=freq[freq!=0]  #   delete 0 element(can't use in logarit function)
        prob=freq/float(np.sum(freq))  
        return -np.sum(prob*np.log(prob)) # apply entropy function
    
    def predict(self,new_data):
        x=new_data
        node=self.root
        label=None
        while node.children:    # recurive until reach leaf node
            if x[node.split_attribute] not in node.val_chosen:
                label='delete'         # prevent if the value of feature not in values of split_attribute since we use random sample
                break
            else:
                node=node.children[node.val_chosen.index(x[node.split_attribute])]
                label=node.label
        return label


In [51]:
def bootstrap(X,y):
    m=X.shape[0]
    ids=np.random.choice(m,m,replace=True)
    return X.iloc[ids,:],y.iloc[ids]



In [102]:
class Random_forest(object):
    def __init__(self,n_tree=10,max_depth=3,min_sample_split=20):
        self.n_tree=n_tree
        self.max_depth=max_depth
        self.min_sample_split=min_sample_split
        self.trees=[]
    def optimizer_step(self,X,y):
        for i in range(self.n_tree):
            tree=decision_tree_id3(max_depth=self.max_depth,min_sample_split=self.min_sample_split)
            X_train,y_train=bootstrap(X,y)
            tree.optimizer_step(X_train,y_train)
            self.trees.append(tree)
    def predict(self,X_test):
        n_point=X_test.count().iloc[0]
        labels=[None]*n_point
        for i in range(n_point):
            x=X_test.iloc[i,:]
            pred=[tree.predict(x) for tree in self.trees]
            label=mode(pred)
            labels[i]=label
        return labels
def arrcuracy(model,X,y):
    yhat=model.predict(X)
    if 'delete' in yhat:
        m=len(yhat.remove('delete'))
    else:
        m=len(yhat)
    sum=np.sum(np.where(y==yhat,1,0))
    return sum/m    

In [111]:
forest=Random_forest(n_tree=20,max_depth=3)
df=pd.read_csv('Breast_Cancer.csv')
m=df.shape[0]
X_train=df.iloc[:int(m*0.6),:-1]
y_train=df.iloc[:int(m*0.6),-1]
X_test=df.iloc[int(m*0.6):,:-1]
y_test=df.iloc[int(m*0.6):,-1]
forest.optimizer_step(X_train,y_train)


In [112]:
print(arrcuracy(forest,X_test,y_test))

0.8503105590062112
